# Goal Build all pinning services into more abstract API to upload and download dataset

* NFT Storage
* Pinata
* Estuary

In [1]:
import pandas as pd

from helpers.helper import read_file

## NFT Storage

In [2]:
from storage.nftstorage import NFTStorage

In [3]:
nft_storage = NFTStorage()

In [4]:
cred = nft_storage.get_creds()

In [8]:
w = open("dataset/sample/tabular-epochs100_ntp5_tw3.pkl","rb")

In [9]:
response,status = nft_storage.upload_file(cred,"dataset/sample/tabular-epochs100_ntp5_tw3.pkl")

In [10]:
check_file = nft_storage.get_file(response["value"]["cid"],cred)

In [11]:
# nft_storage.unpin("bafybeie4yihf42kahspmhqkmovpxt2iliqqikbriwsglb6reujvzsnp5ke",cred)

# IPFS

In [12]:
from storage.ipfs import IPFS
# from helpers.helper import read_file

ipfs = IPFS()

In [18]:
# w = ipfs.add("All_Gateway_Properties.csv")

# w.json()

In [15]:
from storage.ipfs import IPFS
# from helpers.helper import read_file

ipfs = IPFS()

f_name, log = ipfs.get_file(response["value"]["cid"],local_node=False)

import pickle

string_of_bytes_obj = str(pickle.dumps(f_name.content), encoding="latin1")

unpickled_dict = pickle.loads(bytes(string_of_bytes_obj, "latin1"))

import torch
from io import BytesIO

torch.load(BytesIO(unpickled_dict),map_location=torch.device('cpu'))

Retrieved file hash bafybeialu36aw3i2drpvhzc7swsp7rs77wekt5bxmtafypizncltzrymum from https://infura-ipfs.io Response 200


In [14]:
read_file(response)

,Timestamp,Token,Borrowing Rate,Deposit Rate,Borrow Volume,Supply Volume
0,1609471800,DAI,0.073195,0.050982,1.069964e+09,6.196481e+10
1,1609471800,USDC,0.087046,0.066993,7.285430e+08,4.063042e+10
2,1609471800,USDT,0.099588,0.077548,6.430536e+07,3.696225e+09
3,1609471800,ETH,0.022952,0.000489,3.055365e+04,5.663257e+07
4,1609473600,DAI,0.073101,0.050912,1.069961e+09,6.197050e+10
...,...,...,...,...,...,...
66789,1639539000,ETH,0.027215,0.000761,5.316478e+04,7.481990e+07
66790,1639540800,ETH,0.027214,0.000761,5.316480e+04,7.482188e+07
66791,1639542600,ETH,0.027214,0.000761,5.316488e+04,7.482482e+07
66792,1639544400,ETH,0.027226,0.000764,5.316501e+04,7.461519e+07


In [15]:
import pandas as pd
df_peers = pd.DataFrame(ipfs.get_peers()["Peers"]).dropna()

df_peers["Latency"] = df_peers["Latency"].replace("n/a",None)

df_peers["Latency_float"] = df_peers["Latency"].str.strip("ms").astype("float64")

df_peers["Stream_Count"] = df_peers["Streams"].apply(lambda x: len(x))

peers = df_peers["Peer"].to_list()

In [16]:
peers[0]

'12D3KooWKfaq9fFyAyabRywMbTT32jXkDQ7LG13J2tWseXXrebXS'

In [18]:
for peer in peers:

    dht = ipfs.get_dht(peer)
    
    with open(f"dht/dht_{peer}.pkl","w") as f:
    
        f.writelines(dht.text)

In [19]:
import json
import glob

df = pd.DataFrame()

for dht_f in glob.glob("dht/dht_*"):
    
    with open(dht_f,"r") as f:

        d = f.readlines()

        clean = [json.loads(i.strip()) for i in d]
        
        df = pd.concat([df,pd.DataFrame(clean)])

In [20]:
df.to_csv("snapshot1.csv",index=False)

In [ ]:
df_dhtpeers = pd.DataFrame(dht_file,columns=["peers"])

In [ ]:
df_dhtpeers

In [ ]:
ipfs.dht_get_file("bafkreiesap4b4a4xgvfxhsz6hdza7atezclk4op3iz5lusu5xxx7542ns4")

In [ ]:
_ = df_peers["Latency_float"].hist()

In [ ]:
df_peers.describe()

In [ ]:
df_peers

# Pinata

In [17]:
from storage.pinataV1 import PinataV1

pinata = PinataV1()

In [18]:
cred = pinata.get_creds()

In [ ]:
pinata.upload_file("dataset/sample/All_Gateway_Properties.csv",cred)

In [23]:
pinata.pin(cred,response["value"]["cid"],fn="tabular-epochs100_ntp5_tw3.pkl")

({'id': '676bdf01-09a1-4ace-b21d-b825d9200cb9',
  'ipfsHash': 'bafybeialu36aw3i2drpvhzc7swsp7rs77wekt5bxmtafypizncltzrymum',
  'status': 'prechecking',
  'name': 'tabular-epochs100_ntp5_tw3.pkl'},
 200)

In [ ]:
pinata.unpin("QmQwwMbykhX3wyCdC5yBRXV4JQRe5FFLiyynaUiLbmvLzn",cred)

In [ ]:
pinata.edit_hash(cred,"Popcorn Limited - Dune Dashboards - 03182022.pdf","bafkreief472lhr6b54baq4oansiwor4ed2te2xztwgjqez2btrns5d6dxi")

In [ ]:
f_name = ipfs.get_file("QmQwwMbykhX3wyCdC5yBRXV4JQRe5FFLiyynaUiLbmvLzn",local_node=False)


In [ ]:
pinata.pin_policy("bafybeif36uqfqj5on23qfdl3nglc7ncgcbnz4ixpleomskg7pqn3xmlgfi",cred,region="FRA1",replications=1)

In [ ]:
pinata.globalpin_policy(cred,region="FRA1",replications=1)

## Request Files

In [ ]:
import pandas as pd

"""
"sort" - Sort the results by the date added to the pinning queue (see value options below)
"ASC" - Sort by ascending dates
"DESC" - Sort by descending dates
"status" - Filter by the status of the job in the pinning queue (see potential statuses below)
"prechecking" - Pinata is running preliminary validations on your pin request.
"searching" - Pinata is actively searching for your content on the IPFS network. This may take some time if your content is isolated.
"retrieving" - Pinata has located your content and is now in the process of retrieving it.
"expired" - Pinata wasn't able to find your content after a day of searching the IPFS network. Please make sure your content is hosted on the IPFS network before trying to pin again.
"over_free_limit" - Pinning this object would put you over the free tier limit. Please add a credit card to continue pinning content.
"over_max_size" - This object is too large of an item to pin. If you're seeing this, please contact us for a more custom solution.
"invalid_object" - The object you're attempting to pin isn't readable by IPFS nodes. Please contact us if you receive this, as we'd like to better understand what you're attempting to pin.
"bad_host_node" - You provided a host node that was either invalid or unreachable. Please make sure all provided host nodes are online and reachable.
"ipfs_pin_hash" - Retrieve the record for a specific IPFS hash
"limit" - Limit the amount of results returned per page of results (default is 5, and max is 1000)
"offset" - Provide the record offset for records being returned. This is how you retrieve records on additional pages (default is 0)

MetaData
metadata[name]=exampleName

&metadata[keyvalues]={"exampleKey":{"value":"exampleValue","op":"exampleOp"},"exampleKey2":{"value":"exampleValue2","op":"exampleOp2"}}
"""
params = {"sort":"DESC","status":None,"prechecking":None,
          "searching":None,"retrieving":None,
          "expired":None,"over_free_limit":None,
          "over_max_size":None,"invalid_object":None,
          "bad_host_node":None,"ipfs_pin_hash":None,
          "limit":None,"offset":None,"metadata[name]":None,
          "metadatakeyvalues":"keyvalues"
         }

pf,status = pinata.get_pinned_files(cred,params)
pd.DataFrame(pf["rows"])

## Requesting Pin Jobs

In [ ]:
pf,status = pinata.get_pinned_jobs(cred,params)
pf

In [ ]:
pinata.get_datausage(cred)

In [ ]:
#convert bytes to MB

2103065 * 10**-6

# Work in Progress

# Pinata V2

In [ ]:
# import requests module
import requests
import json
from requests.auth import HTTPBasicAuth

class PinataV2:
    
    def get_creds(self):

        with open("creds.json") as f:

            cred = json.loads(f.read())

        return cred["Pinata"]

    def upload_file(self,fn,cred):
        


        url = "https://managed.mypinata.cloud/api/v1/content"

        payload={}
        headers = {
          'x-api-key': cred['API Key']
        }

        response = requests.request("GET", url, headers=headers, data=payload)

        print(response.text)

        return response.json(),response.status_code

In [ ]:
pinatav2 = PinataV2()

In [ ]:
cred = pinatav2.get_creds()

In [ ]:
pinatav2.upload_file("storage/ipfs.py",cred)

# Estuary

In [ ]:
import requests
import json


class Estuary:
    
    def get_creds(self):

        with open("creds.json") as f:

            cred = json.loads(f.read())

        return cred["Estuary"]
    
    def upload_file(self,fn,cred):

        base_url1 = "https://shuttle-4.estuary.tech/content/add"
        base2 = "https://shuttle-5.estuary.tech/content/add"
        
        header = {"Content-Type":"multipart/form-data",
                  "Authorization":"Bearer " + cred["API Key"],
                  }
        
        
        
        #Filename,file, content_type, cookie expiration?
        files = {'file': (fn, open(fn, 'rb'),
                          "multipart/form-data", {'Expires': '0'})
                }



        response = requests.post(base_url1, headers=header,data=files)

        return response.json(),response.status_code
    
    def get_pins(self,cred):

        base_url = "https://api.estuary.tech/pinning/pins"

        header = {"Authorization": "Bearer " + cred["API Key"]}
        
        
        response = requests.get(base_url,headers=header)

        return response.json(),response.status_code





In [ ]:
e = Estuary()

In [ ]:
cred = e.get_creds()

In [ ]:
pins = e.get_pins(cred)

In [ ]:
e.upload_file("dataset/sample/All_Gateway_Properties.csv",cred)